### Compare libem benchmark results from 2 runs

In [17]:
import json

run1_file = '../libem/benchmark/results/???.json'
run2_file = '../libem/benchmark/results/???.json'

runs = {}

with open(run1_file, 'r') as f:
    runs['run1'] = json.load(f)
with open(run2_file, 'r') as f:
    runs['run2'] = json.load(f)

longer_run = 'run1' if len(runs['run1']) >= len(runs['run2']) else 'run2'
shorter_run = 'run1' if len(runs['run1']) < len(runs['run2']) else 'run2'

# hash join using dict
run_dict = {}
for r in runs[shorter_run]:
    entity_info = ' '.join([str(v) for v in r['entity_1'].values()]) + ' '.join([str(v) for v in r['entity_2'].values()])
    run_dict[entity_info] = r
    
    
# process results
results = {
    'run1_only_false_pos': [],
    'run1_only_false_neg': [],
    'run2_only_false_pos': [],
    'run2_only_false_neg': [],
}

found = 0
for r in runs[longer_run]:
    entity_info = ' '.join([str(v) for v in r['entity_1'].values()]) + ' '.join([str(v) for v in r['entity_2'].values()])
    
    if entity_info in run_dict:
        o = run_dict[entity_info]
        found += 1
        
        output = {'entity_1': r['entity_1'], 'entity_2': r['entity_2'], 'tools_used': {longer_run: r.get('tools_used', []), shorter_run: o.get('tools_used', [])}, 'label': r['label']}
        
        if r['pred'][0:3] == 'yes' and not o['pred'][0:3] == 'yes':
            if r['label'] == 1:
                results[f'{shorter_run}_only_false_neg'].append(output)
            else:
                results[f'{longer_run}_only_false_pos'].append(output)
        elif o['pred'][0:3] == 'yes' and not r['pred'][0:3] == 'yes':
            if r['label'] == 0:
                results[f'{shorter_run}_only_false_pos'].append(output)
            else:
                results[f'{longer_run}_only_false_neg'].append(output)

# sanity check: should be the same
print(found, len(runs[shorter_run]))

for k, v in results.items():
    print(k, ":", len(v))

0 250
run1_only_false_pos : 0
run1_only_false_neg : 0
run2_only_false_pos : 0
run2_only_false_neg : 0


In [16]:
from pathlib import Path
Path('./compare_runs_results').mkdir(parents=True, exist_ok=True)

with open(f'./compare_runs_results/{run1_file[27:-5]}_vs_{run2_file[27:-5]}.json', 'w') as f:
    json.dump(results, f, indent=4)